In [121]:
import pandas as pd
import numpy as np
import json
import tqdm

In [122]:
pd.__version__

'1.0.5'

In [123]:
full_movies_df = pd.read_csv("./archive/tmdb_5000_movies.csv")
full_movies_df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


## Movies

In [124]:
movies_df = full_movies_df[['id', 'title', 'tagline', 'overview', 'status', 
                            'release_date', 'original_language', 'budget', 'revenue',
                           'popularity', 'runtime', 'vote_count', 'vote_average', 'homepage']].copy()
movies_df['status'] = movies_df['status'].apply(lambda x: x == 'Released')
movies_df['runtime'] = movies_df['runtime'].astype('Int32')
movies_df.rename(columns={"status": "released"}, inplace=True)
movies_df.head(3)

,id,title,tagline,overview,released,release_date,original_language,budget,revenue,popularity,runtime,vote_count,vote_average,homepage
0,19995,Avatar,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",True,2009-12-10,en,237000000,2787965087,150.437577,162,11800,7.2,http://www.avatarmovie.com/
1,285,Pirates of the Caribbean: At World's End,"At the end of the world, the adventure begins.","Captain Barbossa, long believed to be dead, ha...",True,2007-05-19,en,300000000,961000000,139.082615,169,4500,6.9,http://disney.go.com/disneypictures/pirates/
2,206647,Spectre,A Plan No One Escapes,A cryptic message from Bond’s past sends him o...,True,2015-10-26,en,245000000,880674609,107.376788,148,4466,6.3,http://www.sonypictures.com/movies/spectre/


In [125]:
movies_df.isna().any()

id                   False
title                False
tagline               True
overview              True
released             False
release_date          True
original_language    False
budget               False
revenue              False
popularity           False
runtime               True
vote_count           False
vote_average         False
homepage              True
dtype: bool

In [126]:
def find_str_length(item):
    if type(item) == str:
        return len(item)

    
movies_df['overview'].map(lambda x: find_str_length(x)).max()

1000.0

## Genres

In [127]:
genres_data_json = full_movies_df['genres'].map(lambda x: json.loads(x))
genres_data = genres_data_json.copy()

genres = [genre for genres in genres_data for genre in genres]
unique_genres = list({v['id']:v for v in genres}.values())

genres_df = pd.json_normalize(unique_genres)
genres_df.rename(columns={"name": "genre"}, inplace=True)
genres_df

,id,genre
0,28,Action
1,12,Adventure
2,14,Fantasy
3,878,Science Fiction
4,80,Crime
5,18,Drama
6,53,Thriller
7,16,Animation
8,10751,Family
9,37,Western


In [128]:
genres_count_df = pd.json_normalize(genres)
genres_count_df.rename(columns={"name": "genre"}, inplace=True)
genres_count_df['count'] = genres_count_df.groupby('genre')['genre'].transform('count')
genres_count_df = genres_count_df.drop_duplicates()
genres_count_df = genres_count_df.sort_values(by='count', ascending=False)
genres_count_df = genres_count_df.reset_index(drop=True)
genres_count_df.head(5)

,id,genre,count
0,18,Drama,2297
1,35,Comedy,1722
2,53,Thriller,1274
3,28,Action,1154
4,10749,Romance,894


## Keywords

In [129]:
keywords_data = full_movies_df['keywords'].map(lambda x: json.loads(x))
keywords = [keyword for keywords in keywords_data for keyword in keywords]
unique_keywords = list({v['id']:v for v in keywords}.values())

keywords_df = pd.json_normalize(unique_keywords)
keywords_df.rename(columns={"name": "keyword"}, inplace=True)
keywords_df.head(5)

,id,keyword
0,1463,culture clash
1,2964,future
2,3386,space war
3,3388,space colony
4,3679,society


## Production companies

In [130]:
prodcomp_data = full_movies_df['production_companies'].map(lambda x: json.loads(x))
prodcompanies = [prodcompany for prodcompanies in prodcomp_data for prodcompany in prodcompanies]
unique_companies = list({v['id']:v for v in prodcompanies}.values())

prodcompanies_df = pd.json_normalize(unique_companies)
prodcompanies_df.rename(columns={"name": "company"}, inplace=True)
prodcompanies_df = prodcompanies_df[['id', 'company']]
prodcompanies_df

,id,company
0,289,Ingenious Film Partners
1,306,Twentieth Century Fox Film Corporation
2,444,Dune Entertainment
3,574,Lightstorm Entertainment
4,2,Walt Disney Pictures
...,...,...
5042,26598,FM Entertainment
5043,40739,Eagle Eye Films Inc.
5044,3958,Front Street Pictures
5045,87986,rusty bear entertainment


## Production countries

In [131]:
prodcountries_data = full_movies_df['production_countries'].map(lambda x: json.loads(x))
countries = [country for countries in prodcountries_data for country in countries]
unique_countries = list({v['iso_3166_1']:v for v in countries}.values())

prodcountries_df = pd.json_normalize(unique_countries)
prodcountries_df.reset_index(inplace=True)
prodcountries_df.rename(columns={"index": "id", "iso_3166_1": "code", "name": "country"}, inplace=True)
prodcountries_df['id'] = prodcountries_df['id'].apply(lambda x: x + 1)

prodcountries_df

,id,code,country
0,1,US,United States of America
1,2,GB,United Kingdom
2,3,JM,Jamaica
3,4,BS,Bahamas
4,5,DM,Dominica
...,...,...,...
83,84,AF,Afghanistan
84,85,AO,Angola
85,86,DO,Dominican Republic
86,87,CM,Cameroon


## Spoken languages

In [132]:
languages_data = full_movies_df['spoken_languages'].map(lambda x: json.loads(x))

languages = [language for languages in languages_data for language in languages]
unique_languages = list({v['iso_639_1']:v for v in languages}.values())

spoken_languages_df = pd.json_normalize(unique_languages)
spoken_languages_df.reset_index(inplace=True)
spoken_languages_df.rename(columns={"index": "id", "iso_639_1": "code", "name": "spoken_language"}, inplace=True)
spoken_languages_df['id'] = spoken_languages_df['id'].apply(lambda x: x + 1)
spoken_languages_df

,id,code,spoken_language
0,1,en,English
1,2,es,Español
2,3,fr,Français
3,4,it,Italiano
4,5,de,Deutsch
...,...,...,...
82,83,dz,
83,84,ky,??????
84,85,id,Bahasa indonesia
85,86,bm,Bamanankan


# Relational tables

### MovieGenre

In [133]:
genres = []
for i, genre in enumerate(genres_data_json):
    for g in genre:
        g['movie_id'] = full_movies_df.iloc[i]['id']
        genres.append(g)

movie_genres_df = pd.json_normalize(genres)
movie_genres_df.rename(columns={"id": "genre_id"}, inplace=True)
movie_genres_df.drop(columns={"name"}, inplace=True)
movie_genres_df.reset_index(inplace=True)
movie_genres_df.rename(columns={"index": "id"}, inplace=True)
movie_genres_df['id'] = movie_genres_df['id'].apply(lambda x: x + 1)
movie_genres_df = movie_genres_df[['id', 'movie_id', 'genre_id']]
movie_genres_df

,id,movie_id,genre_id
0,1,19995,28
1,2,19995,12
2,3,19995,14
3,4,19995,878
4,5,285,12
...,...,...,...
12155,12156,231617,35
12156,12157,231617,18
12157,12158,231617,10749
12158,12159,231617,10770


### MovieKeyword

In [134]:
keywords = []
for i, keyword in enumerate(keywords_data):
    for k in keyword:
        k['movie_id'] = full_movies_df.iloc[i]['id']
        keywords.append(k)

movie_keywords_df = pd.json_normalize(keywords)
movie_keywords_df.drop(columns={"name"}, inplace=True)
movie_keywords_df.rename(columns={"id": "keyword_id"}, inplace=True)
movie_keywords_df.reset_index(inplace=True)
movie_keywords_df.rename(columns={"index": "id"}, inplace=True)
movie_keywords_df['id'] = movie_keywords_df['id'].apply(lambda x: x + 1)
movie_keywords_df = movie_keywords_df[['id', 'movie_id', 'keyword_id']]
movie_keywords_df

,id,movie_id,keyword_id
0,1,19995,1463
1,2,19995,2964
2,3,19995,3386
3,4,19995,3388
4,5,19995,3679
...,...,...,...
36189,36190,231617,173066
36190,36191,25975,1523
36191,36192,25975,2249
36192,36193,25975,9986


### MovieProdCompany

In [135]:
companies = []
for i, company in enumerate(prodcomp_data):
    for c in company:
        c['movie_id'] = full_movies_df.iloc[i]['id']
        companies.append(c)

companies_df = pd.json_normalize(companies)
companies_df.drop(columns={"name"}, inplace=True)
companies_df.rename(columns={"id": "company_id"}, inplace=True)
companies_df.reset_index(inplace=True)
companies_df.rename(columns={"index": "id"}, inplace=True)
companies_df['id'] = companies_df['id'].apply(lambda x: x + 1)
companies_df = companies_df[['id', 'movie_id', 'company_id']]
companies_df

,id,movie_id,company_id
0,1,19995,289
1,2,19995,306
2,3,19995,444
3,4,19995,574
4,5,285,2
...,...,...,...
13672,13673,9367,5
13673,13674,231617,3958
13674,13675,231617,6438
13675,13676,25975,87986


### MovieProdCountry

In [136]:
countries = []
for i, country in enumerate(prodcountries_data):
    for c in country:
        c['movie_id'] = full_movies_df.iloc[i]['id']
        countries.append(c)

countries_df = pd.json_normalize(countries)
countries_df.rename(columns={"iso_3166_1": "code"}, inplace=True)
countries_df

,code,name,movie_id
0,US,United States of America,19995
1,GB,United Kingdom,19995
2,US,United States of America,285
3,GB,United Kingdom,206647
4,US,United States of America,206647
...,...,...,...
6431,US,United States of America,9367
6432,US,United States of America,231617
6433,US,United States of America,126186
6434,CN,China,126186


In [137]:
countries_ix = []
for country in countries_df.values:
    countries_ix.append(prodcountries_df[prodcountries_df['code'] == country[0]]['id'].values[0])

countries_df['country_id'] = countries_ix
countries_df.drop(columns={"code", "name"}, inplace=True)
countries_df.reset_index(inplace=True)
countries_df.rename(columns={"index": "id"}, inplace=True)
countries_df['id'] = countries_df['id'].apply(lambda x: x + 1)
countries_df = countries_df[['id', 'movie_id', 'country_id']]
countries_df

,id,movie_id,country_id
0,1,19995,1
1,2,19995,2
2,3,285,1
3,4,206647,2
4,5,206647,1
...,...,...,...
6431,6432,9367,1
6432,6433,231617,1
6433,6434,126186,1
6434,6435,126186,11


### MovieSpokenLanguage

In [138]:
languages = []
for i, language in enumerate(languages_data):
    for l in language:
        l['movie_id'] = full_movies_df.iloc[i]['id']
        languages.append(l)

movie_languages_df = pd.json_normalize(languages)
movie_languages_df.rename(columns={"iso_639_1": "code"}, inplace=True)
movie_languages_df

,code,name,movie_id
0,en,English,19995
1,es,Español,19995
2,en,English,285
3,fr,Français,206647
4,en,English,206647
...,...,...,...
6932,en,English,14337
6933,es,Español,9367
6934,en,English,231617
6935,en,English,126186


In [139]:
languages_ix = []
for language in movie_languages_df.values:
    languages_ix.append(spoken_languages_df[spoken_languages_df['code'] == language[0]]['id'].values[0])

movie_languages_df['language_id'] = languages_ix
movie_languages_df.drop(columns={"code", "name"}, inplace=True)
movie_languages_df.reset_index(inplace=True)
movie_languages_df.rename(columns={"index": "id"}, inplace=True)
movie_languages_df['id'] = movie_languages_df['id'].apply(lambda x: x + 1)
movie_languages_df = movie_languages_df[['id', 'movie_id', 'language_id']]
movie_languages_df

,id,movie_id,language_id
0,1,19995,1
1,2,19995,2
2,3,285,1
3,4,206647,3
4,5,206647,1
...,...,...,...
6932,6933,14337,1
6933,6934,9367,2
6934,6935,231617,1
6935,6936,126186,1


## Save as CSV

In [140]:
basePath = '../dbiola/src/main/resources/data'

In [143]:
movies_df.to_csv(f'{basePath}/independent/movies.csv', index=False)
genres_df.to_csv(f'{basePath}/independent/genres.csv', index=False)
keywords_df.to_csv(f'{basePath}/independent/keywords.csv', index=False)
prodcompanies_df.to_csv(f'{basePath}/independent/production_companies.csv', index=False)
prodcountries_df.to_csv(f'{basePath}/independent/production_countries.csv', index=False)
spoken_languages_df.to_csv(f'{basePath}/independent/spoken_languages.csv', index=False)

In [142]:
movie_genres_df.to_csv(f'{basePath}/relational/movie_genres.csv', index=False)
movie_keywords_df.to_csv(f'{basePath}/relational/movie_keywords.csv', index=False)
companies_df.to_csv(f'{basePath}/relational/movie_prod_companies.csv', index=False)
countries_df.to_csv(f'{basePath}/relational/movie_prod_countries.csv', index=False)
movie_languages_df.to_csv(f'{basePath}/relational/movie_languages.csv', index=False)